In [14]:
import numpy as np
import pandas as pd
import datetime
import math
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import collections
sns.set()

In [16]:
combined_df = pd.read_csv("./Data/question1.csv")

## Emad's Harder Question
### Building Train and Test Dataset to aggregate data

In [17]:
combined_df = combined_df.drop(['error_catagory','name_y','country','fullName', 'resultId'], axis=1)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4786 entries, 0 to 4785
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    4786 non-null   int64  
 1   driverId                  4786 non-null   int64  
 2   constructorId             4786 non-null   int64  
 3   grid                      4786 non-null   int64  
 4   positionOrder             4786 non-null   int64  
 5   statusId                  4786 non-null   int64  
 6   year                      4786 non-null   float64
 7   circuitId                 4786 non-null   float64
 8   turns                     4786 non-null   float64
 9   lap_length                4786 non-null   float64
 10  race_laps                 4786 non-null   float64
 11  race_distance             4786 non-null   float64
 12  max_speed                 4786 non-null   float64
 13  drs_zone                  4786 non-null   float64
 14  full_thr

In [18]:
# Classify each of the downfore level to a number
for index, row in combined_df.iterrows():
    if(row['downforce_level'] == 'High'):
        combined_df.at[index,'downforce_level'] = 3
    elif(row['downforce_level'] == 'Medium'):
        combined_df.at[index,'downforce_level'] = 2
    elif(row['downforce_level'] == 'Low'):
        combined_df.at[index,'downforce_level'] = 1


In [19]:
combined_df['downforce_level'] = combined_df['downforce_level'].astype(str).astype(int)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4786 entries, 0 to 4785
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    4786 non-null   int64  
 1   driverId                  4786 non-null   int64  
 2   constructorId             4786 non-null   int64  
 3   grid                      4786 non-null   int64  
 4   positionOrder             4786 non-null   int64  
 5   statusId                  4786 non-null   int64  
 6   year                      4786 non-null   float64
 7   circuitId                 4786 non-null   float64
 8   turns                     4786 non-null   float64
 9   lap_length                4786 non-null   float64
 10  race_laps                 4786 non-null   float64
 11  race_distance             4786 non-null   float64
 12  max_speed                 4786 non-null   float64
 13  drs_zone                  4786 non-null   float64
 14  full_thr

In [20]:
filtered_combined_no_change = combined_df

In [21]:
grouped_df = filtered_combined_no_change.groupby(['driverId']) 
grouped_df.first()

raceId  constructorId  grid  positionOrder  statusId    year  \
driverId                                                                 
1             18              1     1              1         1  2008.0   
2             18              2     5              2         1  2008.0   
3             18              3     7              3         1  2008.0   
4             18              4    11              4         1  2008.0   
5             18              1     3              5         1  2008.0   
7             18              5    17              7         5  2008.0   
8             18              6    15              8         5  2008.0   
9             18              2     2              9         4  2008.0   
10            18              7    18             10         3  2008.0   
11            18              8    19             11         7  2008.0   
12            18              4    20             12         8  2008.0   
13            18              6     4             13         5  2008.0   
14            18              9     8             14         4  2008.0   
15            18              7     6             15        10  2008.0   
16            18             10    22             16         9  2008.0   
17            18              9    14             17         4  2008.0   
18            18             11    12             18         4  2008.0   
20            18              5     9             20         4  2008.0   
21            18             10    16             21         4  2008.0   
22            18             11    10             22         2  2008.0   
23            36              7     9              8        11  2007.0   
24            36              5    19             14        11  2007.0   
26            36              5    17             21        29  2007.0   
27            36             12    21             22         3  2007.0   
29            46             12    22             22         3  2007.0   
30            53              6     1              2         1  2006.0   
31            53              1     5              5         1  2006.0   
32            53              9     8              8         1  2006.0   
33            53             13    19             17        12  2006.0   
35            53              2    11             20         5  2006.0   
37            63              1     8              7         1  2006.0   
38            68              9    10             12        11  2006.0   
39            71             17    12             15        12  2005.0   
40            71             18    16             17        14  2005.0   
41            79              7    13             17        54  2005.0   
43            90              7    13             12        12  2004.0   
44            90              7    18             13        12  2004.0   
45            90             17    16             14        13  2004.0   
46            91             18    16             14        13  2004.0   
47            90             18    17             19        10  2004.0   
67             1              5    13              7         1  2009.0   
154           11              4    14             15         1  2009.0   
155           16              7    11             10         1  2009.0   
808          337              4    17             21        22  2010.0   
813          841              3    15             20         7  2011.0   
814          841             10    14             10        11  2011.0   
815          841             15    13             22         2  2011.0   
817          849            164    24             19        13  2011.0   
822          880              3    16             14        11  2013.0   
825          900              1     4              2         1  2014.0   
826          900              5     8              9         1  2014.0   
828          900            207    19             17        51  2014.0   
830          9

In [22]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()

for name, group in grouped_df:
    group_size = len(group["constructorId"].unique())
    constructor_arr = group["constructorId"].unique()
    if(group_size == 1):
        train_df = train_df.append(group)
    else:
        for i in range(0,group_size):
            if(i < group_size-1):
                train_df = train_df.append(group[group["constructorId"] == constructor_arr[i]])
            else:
                test_df = test_df.append(group[group["constructorId"] == constructor_arr[i]])

In [23]:
train_df_drop = train_df.dropna()
train_df_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3259 entries, 0 to 4366
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    3259 non-null   int64  
 1   driverId                  3259 non-null   int64  
 2   constructorId             3259 non-null   int64  
 3   grid                      3259 non-null   int64  
 4   positionOrder             3259 non-null   int64  
 5   statusId                  3259 non-null   int64  
 6   year                      3259 non-null   float64
 7   circuitId                 3259 non-null   float64
 8   turns                     3259 non-null   float64
 9   lap_length                3259 non-null   float64
 10  race_laps                 3259 non-null   float64
 11  race_distance             3259 non-null   float64
 12  max_speed                 3259 non-null   float64
 13  drs_zone                  3259 non-null   float64
 14  full_thr

In [24]:
test_df_drop = test_df.dropna()
test_df_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1527 entries, 3283 to 4783
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   raceId                    1527 non-null   int64  
 1   driverId                  1527 non-null   int64  
 2   constructorId             1527 non-null   int64  
 3   grid                      1527 non-null   int64  
 4   positionOrder             1527 non-null   int64  
 5   statusId                  1527 non-null   int64  
 6   year                      1527 non-null   float64
 7   circuitId                 1527 non-null   float64
 8   turns                     1527 non-null   float64
 9   lap_length                1527 non-null   float64
 10  race_laps                 1527 non-null   float64
 11  race_distance             1527 non-null   float64
 12  max_speed                 1527 non-null   float64
 13  drs_zone                  1527 non-null   float64
 14  full_

In [25]:
train_groupby = train_df_drop.groupby(['driverId', 'constructorId', 'year']).agg([np.mean])
train_groupby = train_groupby.reset_index()
train_groupby.info(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   (driverId, )                      203 non-null    int64  
 1   (constructorId, )                 203 non-null    int64  
 2   (year, )                          203 non-null    float64
 3   (raceId, mean)                    203 non-null    float64
 4   (grid, mean)                      203 non-null    float64
 5   (positionOrder, mean)             203 non-null    float64
 6   (statusId, mean)                  203 non-null    float64
 7   (circuitId, mean)                 203 non-null    float64
 8   (turns, mean)                     203 non-null    float64
 9   (lap_length, mean)                203 non-null    float64
 10  (race_laps, mean)                 203 non-null    float64
 11  (race_distance, mean)             203 non-null    float64
 12  (max_spe

In [26]:
test_groupby = test_df_drop.groupby(['driverId', 'constructorId', 'year']).agg([np.mean])
test_groupby = test_groupby.reset_index()
test_groupby.info(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   (driverId, )                      88 non-null     int64  
 1   (constructorId, )                 88 non-null     int64  
 2   (year, )                          88 non-null     float64
 3   (raceId, mean)                    88 non-null     float64
 4   (grid, mean)                      88 non-null     float64
 5   (positionOrder, mean)             88 non-null     float64
 6   (statusId, mean)                  88 non-null     float64
 7   (circuitId, mean)                 88 non-null     float64
 8   (turns, mean)                     88 non-null     float64
 9   (lap_length, mean)                88 non-null     float64
 10  (race_laps, mean)                 88 non-null     float64
 11  (race_distance, mean)             88 non-null     float64
 12  (max_speed

In [27]:
train_groupby.to_csv('./Data/train_q3.csv', index = False)
test_groupby.to_csv('./Data/test_q3.csv', index = False)